In [ ]:
import pandas as pd
import numpy as np
import os, sys
import pickle as pk

In [ ]:
results_file = list(filter(lambda x: "Test.pkl" in x, os.listdir("results")))
results_file = list(filter(lambda x: "MN2" in x, results_file))
results_file.sort()

In [ ]:
rdf = pd.DataFrame(columns=["Method", "Assigment", "Conflict", "Weighted", "Test", "Dataset", "Features", "Classifier", "Accuracy"])

for r in results_file:
    row = dict()
    with open("results/"+r, "rb") as f:
        data = pk.load(f)
    method, assigment, conflict, weighted, test = r.split(".pkl")[0].split("-")
    if weighted != "":
        weighted = False if "No" in weighted else True
    test = 1 if "1" in test else 2
    
    row["Method"] = method
    row["Assigment"] = assigment
    row["Conflict"] = conflict
    row["Weighted"] = weighted
    row["Test"] = test
    
    for dataset, d_result in data.items():
        row["Dataset"] = dataset
        for features, f_result in d_result.items():
            row["Features"] = features
            for classifier, c_result in f_result.items():
                row["Classifier"] = classifier
                row["Accuracy"] = c_result["scores"]["acc"]
                
                rdf.loc[rdf.shape[0]] = row

In [ ]:
rdf_nowiw = rdf[~rdf.Method.str.contains("WiW")]
names_map = dict(zip(rdf_nowiw.Method.unique(), ["CoTrainingByCommittee", 
                                                 "CoForest", 
                                                 "DemocraticCo", 
                                                 "LabelPropagation", 
                                                 "LabelSpreading", 
                                                 "TriTraining"]))
names_map["Default"] = ""
rdf_nowiw = rdf_nowiw.replace(names_map)
rdf_nowiw

In [ ]:
rdf_nowiw["Method_name"] = rdf_nowiw.Method +" - "+ rdf_nowiw.Classifier
rdf_nowiw["Dataset_name"] = rdf_nowiw.Dataset + "_" + rdf_nowiw.Features
rdf_nowiw = rdf_nowiw.replace({'Method_name': r' - $'}, {'Method_name': ""}, regex=True)
rdf_nowiw = rdf_nowiw.sort_values(by="Dataset_name")
rdf_nowiw = rdf_nowiw.drop(["Assigment", "Conflict", "Weighted", "Dataset", "Method", "Features", "Classifier"], axis=1)

In [ ]:
rdf1 = rdf_nowiw[rdf_nowiw.Test==2]
rdf2 = rdf_nowiw[rdf_nowiw.Test==1]

In [ ]:
csv1 = rdf1.pivot(index="Method_name", columns="Dataset_name", values="Accuracy").sort_index()
csv2 = rdf2.pivot(index="Method_name", columns="Dataset_name", values="Accuracy")

In [ ]:
csv1.to_csv("results/transductive_acc_train1half_mn2.csv")
csv2.to_csv("results/transductive_acc_train2half_mn2.csv")
csv11 = pd.read_csv("results/transductive_acc_train1half.csv", index_col="Method_name")
csv22 = pd.read_csv("results/transductive_acc_train2half.csv", index_col="Method_name")
pd.concat((csv1, csv11), axis=1).reindex(sorted(list(csv1.columns)+list(csv11.columns)), axis=1).to_csv("results/transductive_firstHalf.csv")
pd.concat((csv2, csv22), axis=1).reindex(sorted(list(csv1.columns)+list(csv11.columns)), axis=1).to_csv("results/transductive_secondHalf.csv")
pd.concat((csv2, csv22), axis=1).reindex(sorted(list(csv1.columns)+list(csv11.columns)), axis=1)
((pd.concat((csv1, csv11), axis=1) + pd.concat((csv2, csv22), axis=1))/2).to_csv("results/transductive_avg.csv")
((pd.concat((csv1, csv11), axis=1) + pd.concat((csv2, csv22), axis=1))/2).reindex(sorted(list(csv1.columns)+list(csv11.columns)), axis=1).to_csv("results/transductive_avg.csv")